In [9]:
import requests
import base64
import json
import xmltodict

from datetime import datetime
from urllib.parse import unquote
from typing import Dict, List, Optional, Union

CLIENT_ID = "TomArund-ReadInve-PRD-e66f79d08-8f8acc4d"
CLIENT_SECRET = "PRD-66f79d08e8ad-bf57-4662-affd-8697"
RU_NAME = "Tom_Arundel-TomArund-ReadIn-zrmym"
ENCODED_AUTH_CODE = "v%5E1.1%23i%5E1%23p%5E3%23r%5E1%23I%5E3%23f%5E0%23t%5EUl41XzM6MEU4NTBGRTJGMkU3NDE5MzFCMDk1QkRCOEREQzg4QUFfMF8xI0VeMjYw"
# REFRESH_TOKEN = "v^1.1#i^1#f^0#I^3#r^1#p^3#t^Ul4xMF8xMTozNDU2NUMwRUI1QTBBNzEwQTQ3QzU3ODM0MkI2NjM0Ml8xXzEjRV4yNjA="

AUTH_CODE = unquote(ENCODED_AUTH_CODE)

CLIENT_ID_SANDBOX = "TomArund-ReadInve-SBX-9670c09d3-9e27b8bf"
CLIENT_SECRET_SANDBOX = "SBX-670c09d36536-aa58-4f23-9bf4-53c8"
RU_NAME_SANDBOX = "Tom_Arundel-TomArund-ReadIn-xqbypnfx"

In [3]:
# This is the URL required in order to get a new refresh token (1x every 2 years or 'refresh_token_expires_in': 47304000,). 
# Need to log when this is to anticipate when we need to refresh.
# YOU OPEN THE URL THAT IS CREATED AND GRAB THE TOKEN FROM THE URL IN BROWSER SEARCH BAR - CURRENTLY MANUAL PROCESS

url = f"https://auth.ebay.com/oauth2/authorize?client_id={CLIENT_ID}&redirect_uri={RU_NAME}&response_type=code&scope=https://api.ebay.com/oauth/api_scope/sell.inventory"

print(url)

https://auth.ebay.com/oauth2/authorize?client_id=TomArund-ReadInve-PRD-e66f79d08-8f8acc4d&redirect_uri=Tom_Arundel-TomArund-ReadIn-zrmym&response_type=code&scope=https://api.ebay.com/oauth/api_scope/sell.inventory


In [6]:
# Code up to only run if refresh token > 1 year to go

def get_refresh_token():

    # Step 1: Prepare the request
    token_url = "https://api.ebay.com/identity/v1/oauth2/token"
    auth_header = base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode()).decode()

    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Authorization": f"Basic {auth_header}"
    }

    data = {
        "grant_type": "authorization_code",
        "code": AUTH_CODE,
        "redirect_uri": RU_NAME
    }

    # Step 2: Make the request
    response = requests.post(token_url, headers=headers, data=data)

    # Step 3: Handle the response
    if response.status_code == 200:
        tokens = response.json()
        print("Access Token:", tokens["access_token"])
        print("Refresh Token:", tokens["refresh_token"])

        print("Expires In:", tokens["expires_in"], "seconds")
        return tokens
    else:
        print("Error:", response.status_code)
        print(response.text)

tokens = get_refresh_token()

Access Token: v^1.1#i^1#p^3#r^0#f^0#I^3#t^H4sIAAAAAAAAAOVZbWwcxRn22Y7BpHYUCCWKqLgeBaSEvZvdvd3b3fqOru/WySU+3+Xu7MSBcNqP2fPG+3HenbV96ZexKFTqn7SUUgRRg0QlvlIhklBB1QiBQCFA8qOlqmhaQGqh8AMUlSpKWyXdPduJ4zZJm3Xblbo66TSz7868z/N+zDszYKare/19m+471RO5qn3fDJhpj0TwlaC7a8WG3o72dSvawCKByL6ZL810znb8sc8RDb3BlaHTsEwHRqcN3XS4Vmc65tomZ4mO5nCmaECHQzJX4QuDHBEHXMO2kCVbeiyaz6VjJEPgtKSmWCVFQhmKXq+5MGbVSsdUIqmwEiUyJCUxKUh77x3HhXnTQaKJ0jECEBQGCIygqjjJEYAjmThBkjti0RFoO5pleiJxEMu01OVa39qLdL20qqLjQBt5g8QyeX6gUuTzOWGo2pdYNFZmnocKEpHrXNjKWgqMjoi6Cy89jdOS5iquLEPHiSUyczNcOCjHLyhzBeq3qJYViVJUQoUyAVkpSSwLlQOWbYjo0nr4PZqCqS1RDppIQ83LMeqxIe2CMppvDXlD5HNR/2+rK+qaqkE7HRP6+dHhilCORSulkm1NagpUfKQ4S6e8H0PEMsgyjGYNWZZuiOb8PHODzbO8ZKKsZSqaz5kTHbJQP/SUhkupwRdR4wkVzaLNq8hXaLEctUAhQe/wbTpnRBeNmb5ZoeHxEG01L2+ABY847wPL5ROpFC6lvLijVFKipeQ/cQk/1v9tt8j4luFLpYSvCpTEJmaI9jhEDV2UISZ77LoGtDWFIymVIBkVYgrNqliSVVXMc1Iaw1UIAYSSJLPM/5F3IGRrkovgOQ9Z+qKFMR2ryFYDlixdk5uxpSKthDPvD9NOOjaGUINLJKampuJTZNyy6wkCADyxvTBYkceg4WXcBVnt8sKY1nINGXpfORqHmg1Pm2nP8b

In [12]:
tokens['access_token']

'v^1.1#i^1#p^3#r^0#f^0#I^3#t^H4sIAAAAAAAAAOVZbWwcxRn22Y7BpHYUCCWKqLgeBaSEvZvdvd3b3fqOru/WySU+3+Xu7MSBcNqP2fPG+3HenbV96ZexKFTqn7SUUgRRg0QlvlIhklBB1QiBQCFA8qOlqmhaQGqh8AMUlSpKWyXdPduJ4zZJm3Xblbo66TSz7868z/N+zDszYKare/19m+471RO5qn3fDJhpj0TwlaC7a8WG3o72dSvawCKByL6ZL810znb8sc8RDb3BlaHTsEwHRqcN3XS4Vmc65tomZ4mO5nCmaECHQzJX4QuDHBEHXMO2kCVbeiyaz6VjJEPgtKSmWCVFQhmKXq+5MGbVSsdUIqmwEiUyJCUxKUh77x3HhXnTQaKJ0jECEBQGCIygqjjJEYAjmThBkjti0RFoO5pleiJxEMu01OVa39qLdL20qqLjQBt5g8QyeX6gUuTzOWGo2pdYNFZmnocKEpHrXNjKWgqMjoi6Cy89jdOS5iquLEPHiSUyczNcOCjHLyhzBeq3qJYViVJUQoUyAVkpSSwLlQOWbYjo0nr4PZqCqS1RDppIQ83LMeqxIe2CMppvDXlD5HNR/2+rK+qaqkE7HRP6+dHhilCORSulkm1NagpUfKQ4S6e8H0PEMsgyjGYNWZZuiOb8PHODzbO8ZKKsZSqaz5kTHbJQP/SUhkupwRdR4wkVzaLNq8hXaLEctUAhQe/wbTpnRBeNmb5ZoeHxEG01L2+ABY847wPL5ROpFC6lvLijVFKipeQ/cQk/1v9tt8j4luFLpYSvCpTEJmaI9jhEDV2UISZ77LoGtDWFIymVIBkVYgrNqliSVVXMc1Iaw1UIAYSSJLPM/5F3IGRrkovgOQ9Z+qKFMR2ryFYDlixdk5uxpSKthDPvD9NOOjaGUINLJKampuJTZNyy6wkCADyxvTBYkceg4WXcBVnt8sKY1nINGXpfORqHmg1Pm2nP8bzJzXosQ9pKSbR

In [13]:
# Your client credentials
client_id = CLIENT_ID
client_secret = CLIENT_SECRET

# Base64 encode the client ID and client secret
credentials = f"{client_id}:{client_secret}"
encoded_credentials = base64.b64encode(credentials.encode('utf-8')).decode('utf-8')

# Set up the endpoint and headers
url = 'https://api.ebay.com/identity/v1/oauth2/token'

headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Authorization': f'Basic {encoded_credentials}'
}

# Set up the data payload
data = {
    'grant_type': 'refresh_token',
    'refresh_token': tokens['refresh_token'],
    'scope': 'https://api.ebay.com/oauth/api_scope/sell.inventory'
}

# Make the POST request
response = requests.post(url, headers=headers, data=data)

# Print the response
access_data = response.json()

In [14]:
access_data

{'access_token': 'v^1.1#i^1#p^3#I^3#r^0#f^0#t^H4sIAAAAAAAAAOVZe4wbRxm37xUuaS48wqMNIGfbUuhp7dld73q9xC7r817i5Hz2+ZHkIhV3vDvr29w+nN3Zu/NJSNcjhJQ3tBUgFSmqgqgqKChBoEhQVVVVyh9VSkPVAKJtSEqpyqMSFBAIyq7vcrkcJVG7B6zAsmTP7Lcz3+/3PeabGbAwMHjzkV1H/rQ5uqHn2AJY6IlGqU1gcKB/eKi357r+CFglED22cMNC32Lvr3Y40NDbQgU5bct0UGzO0E1H6HZmCNc2BQs6miOY0ECOgGWhKhbHBDoOhLZtYUu2dCJWyGcIBQIGsKkUpaRSaorjvF7z4pg1K0NwLJNKQTmpeH8Qg4D33HFcVDAdDE2cIWhAsySgSZqtUYxAswLg4izFHSBie5HtaJbpicQBke2qK3TftVfpemVVoeMgG3uDENmCOFotiYW8NF7bkVg1VnaZhyqG2HUub41YCorthbqLrjyN05UWqq4sI8chEtmlGS4fVBAvKvMG1O9SjUAashRNUUmWB0l6XZgctWwD4iur4fdoCql2RQVkYg13rkaoR0bzIJLxcmvcG6KQj/k/Ey7UNVVDdoaQcuJkvSpViFi1XLatGU1Big+USnMp78vTRBZbhtFpYMvSDWguz7M02DLJayYasUxF8ylzYuMWziFPabSWGmoVNZ5QySzZoop9hVbLsSsU0gd8ky7Z0MVTpm9VZHg8xLrNqxvgokNccoF1cwkaAA7SHANhk4JK8jV8wo/11+0XWd80Yrmc8HVBTdghDWhPI9zWoYxI2aPXNZCtKQLDqjTDq4hUuLRKJtOqSjZZhSMpFSGAULMpp/n/I/fA2NaaLkYrLrL2QRdjhqjKVhuVLV2TO8RakW7CWXaIOSdDTGHcFhKJ2dnZ+CwTt+xWwjM6ldhfHKvKU8iAxIqsdnVhUuu6hoy8txxNwJ22p82c53ne5GaLyD

In [19]:

def get_active_listings(auth_token):

    endpoint = 'https://api.ebay.com/ws/api.dll'

    headers = {
        'X-EBAY-API-CALL-NAME': 'GetMyeBaySelling',
        'X-EBAY-API-SITEID': '3',  # UK=3, US=0
        'X-EBAY-API-COMPATIBILITY-LEVEL': '1155',
        'X-EBAY-API-IAF-TOKEN': auth_token,
        'Content-Type': 'text/xml'
    }

    # XML request body
    xml_request = f"""<?xml version="1.0" encoding="utf-8"?>
    <GetMyeBaySellingRequest xmlns="urn:ebay:apis:eBLBaseComponents">
        <RequesterCredentials>
            <eBayAuthToken>{auth_token}</eBayAuthToken>
        </RequesterCredentials>
        <ActiveList>
            <Include>true</Include>
            <Pagination>
                <EntriesPerPage>200</EntriesPerPage>
                <PageNumber>1</PageNumber>
            </Pagination>
        </ActiveList>
        <DetailLevel>ReturnAll</DetailLevel>
    </GetMyeBaySellingRequest>"""

    try:
        response = requests.post(endpoint, headers=headers, data=xml_request)

        # Convert XML response to dictionary/JSON
        response_dict = xmltodict.parse(response.content)

        # Convert to more manageable JSON structure
        listings = []
        active_list = response_dict.get('GetMyeBaySellingResponse', {}).get('ActiveList', {})
        items = active_list.get('ItemArray', {}).get('Item', [])

        # Handle single item case
        if not isinstance(items, list):
            items = [items]

        for item in items:
            listing = {
                'ItemID': item.get('ItemID'),
                'Title': item.get('Title'),
                'Price': item.get('SellingStatus', {}).get('CurrentPrice', {}).get('@currencyID'),
                'Amount': item.get('SellingStatus', {}).get('CurrentPrice', {}).get('#text'),
                'Quantity': item.get('Quantity'),
                'QuantityAvailable': item.get('QuantityAvailable'),
                'ListingStatus': item.get('SellingStatus', {}).get('ListingStatus'),
                'ViewItemURL': item.get('ListingDetails', {}).get('ViewItemURL'),
                # Add category information
                'PrimaryCategoryID': item.get('PrimaryCategory', {}).get('CategoryID'),
                'PrimaryCategoryName': item.get('PrimaryCategory', {}).get('CategoryName'),
                'SecondaryCategoryID': item.get('SecondaryCategory', {}).get('CategoryID'),
                'SecondaryCategoryName': item.get('SecondaryCategory', {}).get('CategoryName')
            }
            listings.append(listing)

        return listings

    except Exception as e:
        print(f"Error: {str(e)}")
        return None

listings = get_active_listings(auth_token=access_data['access_token'])

if listings:
    for listing in listings:
        print(f"Title: {listing['Title']}")
        print(f"Price: {listing['Amount']} {listing['Price']}")
        print(f"Item ID: {listing['ItemID']}")
        print("-------------------")


Title: Apple iPhone 7 - 32GB - Rose Gold (BT Mobile) A1778 (GSM)
Price: 449.99 GBP
Item ID: 167279800027
-------------------


In [18]:
def update_listing(auth_token, item_id, updates):
    endpoint = 'https://api.ebay.com/ws/api.dll'

    headers = {
        'X-EBAY-API-CALL-NAME': 'ReviseItem',
        'X-EBAY-API-SITEID': '3',  # UK=3, US=0
        'X-EBAY-API-COMPATIBILITY-LEVEL': '1155',
        'X-EBAY-API-IAF-TOKEN': auth_token,
        'Content-Type': 'text/xml'
    }

    # Build the update fields XML
    update_fields = ""
    if 'Title' in updates:
        update_fields += f"<Title>{updates['Title']}</Title>"
    if 'Description' in updates:
        update_fields += f"<Description>{updates['Description']}</Description>"
    if 'Price' in updates:
        update_fields += f"<StartPrice>{updates['Price']}</StartPrice>"
    if 'Quantity' in updates:
        update_fields += f"<Quantity>{updates['Quantity']}</Quantity>"

    xml_request = f"""<?xml version="1.0" encoding="utf-8"?>
    <ReviseItemRequest xmlns="urn:ebay:apis:eBLBaseComponents">
        <RequesterCredentials>
            <eBayAuthToken>{auth_token}</eBayAuthToken>
        </RequesterCredentials>
        <Item>
            <ItemID>{item_id}</ItemID>
            {update_fields}
        </Item>
    </ReviseItemRequest>"""

    try:
        response = requests.post(endpoint, headers=headers, data=xml_request)
        response_dict = xmltodict.parse(response.content)
        return response_dict
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

updates = {
    'Price': '449.99',
    'Quantity': '1'
}

item_id = "167279800027"
result = update_listing(access_data['access_token'], item_id, updates)
result

{'ReviseItemResponse': {'@xmlns': 'urn:ebay:apis:eBLBaseComponents',
  'Timestamp': '2025-02-25T13:26:45.898Z',
  'Ack': 'Warning',
  'Errors': [{'ShortMessage': 'Funds from your sales may be unavailable and show as on hold for a period of time.',
    'LongMessage': 'Funds from your sales may be unavailable and show as on hold for a period of time. Learn more:https://www.ebay.co.uk/help/selling/getting-paid/getting-paid-items-youve-sold/pending-payments?id=4816',
    'ErrorCode': '21917236',
    'SeverityCode': 'Warning',
    'ErrorParameters': [{'@ParamID': '0',
      'Value': 'Funds from your sales may be unavailable and show as on hold for a period of time.'},
     {'@ParamID': '1',
      'Value': 'Funds from your sales may be unavailable and show as on hold for a period of time. Learn more:https://www.ebay.co.uk/help/selling/getting-paid/getting-paid-items-youve-sold/pending-payments?id=4816'},
     {'@ParamID': '2', 'Value': 'PayPalHold'},
     {'@ParamID': '3',
      'Value': 'ht

In [16]:
# Update just the price
result = update_listing(auth_token, item_id, {'Price': '25.99'})

# Update title and quantity
result = update_listing(auth_token, item_id, {
    'Title': 'Updated Product Name',
    'Quantity': '10'
})

NameError: name 'auth_token' is not defined

In [17]:

class EbayTrader:
    def __init__(self, auth_token: str, site_id: str = '3'):
        self.auth_token = auth_token
        self.site_id = site_id  # Default to UK (3)
        self.endpoint = 'https://api.ebay.com/ws/api.dll'
        self.compatibility_level = '1155'

    def _make_request(self, call_name: str, xml_request: str) -> Dict:
        """Make request to eBay Trading API"""
        headers = {
            'X-EBAY-API-CALL-NAME': call_name,
            'X-EBAY-API-SITEID': self.site_id,
            'X-EBAY-API-COMPATIBILITY-LEVEL': self.compatibility_level,
            'X-EBAY-API-IAF-TOKEN': self.auth_token,
            'Content-Type': 'text/xml'
        }
        
        try:
            response = requests.post(self.endpoint, headers=headers, data=xml_request)
            return xmltodict.parse(response.content)
        except Exception as e:
            print(f"Error in API call {call_name}: {str(e)}")
            return {}

    def get_item_details(self, item_id: str) -> Dict:
        """Get detailed information for a specific item"""
        xml_request = f"""<?xml version="1.0" encoding="utf-8"?>
    <GetItemRequest xmlns="urn:ebay:apis:eBLBaseComponents">
        <RequesterCredentials>
            <eBayAuthToken>{self.auth_token}</eBayAuthToken>
        </RequesterCredentials>
        <ItemID>{item_id}</ItemID>
        <DetailLevel>ReturnAll</DetailLevel>
    </GetItemRequest>"""
        
        response = self._make_request('GetItem', xml_request)
        
        # Debug print for shipping details
        if response and 'GetItemResponse' in response:
            item = response['GetItemResponse'].get('Item', {})
            print("\nFull Shipping Details from existing item:")
            print(item.get('ShippingDetails', {}))
        
        return response

    def get_active_listings(self, page_num: int = 1, items_per_page: int = 200, include_details: bool = True) -> List[Dict]:
        """Get all active listings"""
        xml_request = f"""<?xml version="1.0" encoding="utf-8"?>
        <GetMyeBaySellingRequest xmlns="urn:ebay:apis:eBLBaseComponents">
            <RequesterCredentials>
                <eBayAuthToken>{self.auth_token}</eBayAuthToken>
            </RequesterCredentials>
            <ActiveList>
                <Include>true</Include>
                <Pagination>
                    <EntriesPerPage>{items_per_page}</EntriesPerPage>
                    <PageNumber>{page_num}</PageNumber>
                </Pagination>
            </ActiveList>
            <DetailLevel>ReturnAll</DetailLevel>
        </GetMyeBaySellingRequest>"""
    
        response_dict = self._make_request('GetMyeBaySelling', xml_request)
        
        listings = []
        active_list = response_dict.get('GetMyeBaySellingResponse', {}).get('ActiveList', {})
        items = active_list.get('ItemArray', {}).get('Item', [])
        
        if not isinstance(items, list):
            items = [items]
            
        for item in items:
            listing = {
                'ItemID': item.get('ItemID'),
                'Title': item.get('Title'),
                'Price': item.get('SellingStatus', {}).get('CurrentPrice', {}).get('@currencyID'),
                'Amount': item.get('SellingStatus', {}).get('CurrentPrice', {}).get('#text'),
                'Quantity': item.get('Quantity'),
                'QuantityAvailable': item.get('QuantityAvailable'),
                'ListingStatus': item.get('SellingStatus', {}).get('ListingStatus'),
                'ViewItemURL': item.get('ListingDetails', {}).get('ViewItemURL'),
                'GalleryURL': item.get('PictureDetails', {}).get('GalleryURL'),
                'StartTime': item.get('ListingDetails', {}).get('StartTime'),
                'TimeLeft': item.get('TimeLeft'),
                'ShippingCost': item.get('ShippingDetails', {}).get('ShippingServiceOptions', {}).get('ShippingServiceCost', {}).get('#text')
            }
            
            # Get additional details if requested
            if include_details:
                item_details = self.get_item_details(item['ItemID'])
                item_data = item_details.get('GetItemResponse', {}).get('Item', {})
                
                listing.update({
                    'PrimaryCategoryID': item_data.get('PrimaryCategory', {}).get('CategoryID'),
                    'PrimaryCategoryName': item_data.get('PrimaryCategory', {}).get('CategoryName'),
                    'Description': item_data.get('Description'),
                    'PictureURLs': item_data.get('PictureDetails', {}).get('PictureURL', []),
                    'PaymentMethods': item_data.get('PaymentMethods', []),
                    'Location': item_data.get('Location'),
                    'Country': item_data.get('Country'),
                    'ConditionID': item_data.get('ConditionID'),
                    'ConditionDisplayName': item_data.get('ConditionDisplayName')
                })
                
            listings.append(listing)
                
        return listings

    def create_listing(self, item_details: Dict) -> Dict:
       # Create picture details XML from URLs
       picture_details = ""
       if 'photos' in item_details and item_details['photos']:
           picture_details = "<PictureDetails>"
           for photo_url in item_details['photos']:
               picture_details += f"<PictureURL>{photo_url}</PictureURL>"
           picture_details += "</PictureDetails>"

       xml_request = f"""<?xml version="1.0" encoding="utf-8"?>
    <AddItemRequest xmlns="urn:ebay:apis:eBLBaseComponents">
       <RequesterCredentials>
           <eBayAuthToken>{self.auth_token}</eBayAuthToken>
       </RequesterCredentials>
       <Item>
           <Title>{item_details['Title']}</Title>
           <Description>{item_details['Description']}</Description>
           <PrimaryCategory>
               <CategoryID>{item_details['CategoryID']}</CategoryID>
           </PrimaryCategory>
           {picture_details}
           <ItemSpecifics>
               <NameValueList>
                   <Name>Brand</Name>
                   <Value>Apple</Value>
               </NameValueList>
               <NameValueList>
                   <Name>Model</Name>
                   <Value>iPhone 7</Value>
               </NameValueList>
               <NameValueList>
                   <Name>Storage Capacity</Name>
                   <Value>32 GB</Value>
               </NameValueList>
               <NameValueList>
                   <Name>Colour</Name>
                   <Value>Rose Gold</Value>
               </NameValueList>
           </ItemSpecifics>
           <StartPrice currencyID="{item_details.get('Currency', 'GBP')}">{item_details['Price']}</StartPrice>
           <Quantity>{item_details['Quantity']}</Quantity>
           <ConditionID>{item_details['ConditionID']}</ConditionID>
           <Location>{item_details.get('Location', 'London')}</Location>
           <Country>{item_details.get('Country', 'GB')}</Country>
           <Currency>{item_details.get('Currency', 'GBP')}</Currency>
           <ListingDuration>GTC</ListingDuration>
           <ListingType>FixedPriceItem</ListingType>

           <ReturnPolicy>
               <ReturnsAcceptedOption>{item_details.get('ReturnsAcceptedOption', 'ReturnsAccepted')}</ReturnsAcceptedOption>
               <ReturnsWithinOption>{item_details.get('ReturnsWithinOption', 'Days_30')}</ReturnsWithinOption>
               <ShippingCostPaidByOption>{item_details.get('ShippingCostPaidByOption', 'Buyer')}</ShippingCostPaidByOption>
           </ReturnPolicy>
           
           <ShippingDetails>
               <ShippingType>Flat</ShippingType>
               <ShippingServiceOptions>
                   <ShippingService>UK_myHermesDoorToDoorService</ShippingService>
                   <ShippingServiceCost currencyID="GBP">{item_details.get('ShippingCost', '2.99')}</ShippingServiceCost>
                   <ShippingServiceAdditionalCost currencyID="GBP">0.0</ShippingServiceAdditionalCost>
                   <ShippingServicePriority>1</ShippingServicePriority>
                   <ExpeditedService>false</ExpeditedService>
                   <ShippingTimeMin>2</ShippingTimeMin>
                   <ShippingTimeMax>3</ShippingTimeMax>
               </ShippingServiceOptions>
           </ShippingDetails>
           <DispatchTimeMax>3</DispatchTimeMax>
       </Item>
    </AddItemRequest>"""
    
       return self._make_request('AddItem', xml_request)

    def update_listing(self, item_id: str, updates: Dict) -> Dict:
        """Update an existing listing"""
        update_fields = ""
        for key, value in updates.items():
            update_fields += f"<{key}>{value}</{key}>"

        xml_request = f"""<?xml version="1.0" encoding="utf-8"?>
        <ReviseItemRequest xmlns="urn:ebay:apis:eBLBaseComponents">
            <RequesterCredentials>
                <eBayAuthToken>{self.auth_token}</eBayAuthToken>
            </RequesterCredentials>
            <Item>
                <ItemID>{item_id}</ItemID>
                {update_fields}
            </Item>
        </ReviseItemRequest>"""

        return self._make_request('ReviseItem', xml_request)

    def end_listing(self, item_id: str, reason: str = 'NotAvailable') -> Dict:
        """End a listing early"""
        xml_request = f"""<?xml version="1.0" encoding="utf-8"?>
        <EndItemRequest xmlns="urn:ebay:apis:eBLBaseComponents">
            <RequesterCredentials>
                <eBayAuthToken>{self.auth_token}</eBayAuthToken>
            </RequesterCredentials>
            <ItemID>{item_id}</ItemID>
            <EndingReason>{reason}</EndingReason>
        </EndItemRequest>"""

        return self._make_request('EndItem', xml_request)

    def mark_as_sold(self, item_id: str) -> Dict:
        """Mark item as sold by ending listing with SoldOffeBay reason"""
        return self.end_listing(item_id, reason='SoldOffeBay')

    def get_condition_values(self, category_id: str) -> List[Dict]:
        """Get valid condition IDs and names for a specific category"""
        xml_request = f"""<?xml version="1.0" encoding="utf-8"?>
        <GetCategoryFeaturesRequest xmlns="urn:ebay:apis:eBLBaseComponents">
            <RequesterCredentials>
                <eBayAuthToken>{self.auth_token}</eBayAuthToken>
            </RequesterCredentials>
            <CategoryID>{category_id}</CategoryID>
            <FeatureID>ConditionValues</FeatureID>
            <DetailLevel>ReturnAll</DetailLevel>
        </GetCategoryFeaturesRequest>"""

        response = self._make_request('GetCategoryFeatures', xml_request)
        category = response.get('GetCategoryFeaturesResponse', {}).get('Category', {})
        conditions = category.get('ConditionValues', {}).get('Condition', [])

        if not isinstance(conditions, list):
            conditions = [conditions]

        return [{
            'ID': condition.get('ID'),
            'DisplayName': condition.get('DisplayName'),
            'Description': condition.get('Description')
        } for condition in conditions]

    def get_shipping_services(self, country_code: str = 'GB') -> List[Dict]:
        """Get available shipping services"""
        xml_request = f"""<?xml version="1.0" encoding="utf-8"?>
        <GeteBayDetailsRequest xmlns="urn:ebay:apis:eBLBaseComponents">
            <RequesterCredentials>
                <eBayAuthToken>{self.auth_token}</eBayAuthToken>
            </RequesterCredentials>
            <DetailName>ShippingServiceDetails</DetailName>
        </GeteBayDetailsRequest>"""
    
        response = self._make_request('GeteBayDetails', xml_request)
        services = response.get('GeteBayDetailsResponse', {}).get('ShippingServiceDetails', [])
        
        if not isinstance(services, list):
            services = [services]
        
        return [{
            'ServiceID': service.get('ShippingService'),
            'Description': service.get('Description'),
            'ServiceType': service.get('ServiceType'),
            'ValidForSellingFlow': service.get('ValidForSellingFlow') == 'true'
        } for service in services if service.get('ValidForSellingFlow') == 'true']

    def get_category_specifics(self, category_id: str) -> Dict:
        """Get required and optional item specifics for a category"""
        xml_request = f"""<?xml version="1.0" encoding="utf-8"?><GetCategorySpecificsRequest xmlns="urn:ebay:apis:eBLBaseComponents"><RequesterCredentials><eBayAuthToken>{self.auth_token}</eBayAuthToken></RequesterCredentials><CategoryID>{category_id}</CategoryID></GetCategorySpecificsRequest>"""

        print("Attempting with completely flat XML...")
        print(xml_request)

        try:
            response = self._make_request('GetCategorySpecifics', xml_request)
            print("Raw API Response:")
            print(response)

            if not response:
                return {}

            category_specifics = response.get('GetCategorySpecificsResponse', {})
            print("Category Specifics Response:")
            print(category_specifics)

            return category_specifics
        except Exception as e:
            print(f"Exception details: {str(e)}")
            return {}

            
    def get_listing_requirements(self, category_id: str) -> Dict:
        """Get listing requirements for a specific category"""
        mandatory_fields = [
            'Title',          # 80 characters max
            'Description',    # HTML allowed
            'CategoryID',     # Must be a valid category ID
            'Price',         # Starting price
            'Quantity',      # Number of items
            'ConditionID',   # Must be valid for category
            'Location',      # Item location
            'Country',       # Country code (e.g., GB)
            'Currency',      # e.g., GBP
            'ListingDuration', # e.g., Days_7, GTC
            'ListingType',    # FixedPriceItem or Auction
            'PaymentMethods', # e.g., PayPal
            'ReturnPolicy',   # Must include return acceptance, period, etc.
            'ShippingDetails' # Must include at least one shipping service
        ]

        # Get category-specific requirements
        xml_request = f"""<?xml version="1.0" encoding="utf-8"?>
        <GetCategoryFeaturesRequest xmlns="urn:ebay:apis:eBLBaseComponents">
            <RequesterCredentials>
                <eBayAuthToken>{self.auth_token}</eBayAuthToken>
            </RequesterCredentials>
            <CategoryID>{category_id}</CategoryID>
            <DetailLevel>ReturnAll</DetailLevel>
        </GetCategoryFeaturesRequest>"""
    
        response = self._make_request('GetCategoryFeatures', xml_request)
        category_features = response.get('GetCategoryFeaturesResponse', {}).get('Category', {})
    
        return {
            'mandatory_fields': mandatory_fields,
            'category_features': category_features,
            'listing_durations': {
                'FixedPriceItem': ['Days_3', 'Days_5', 'Days_7', 'Days_10', 'Days_30', 'GTC'],
                'Auction': ['Days_1', 'Days_3', 'Days_5', 'Days_7', 'Days_10']
            },
            'return_policy_options': {
                'returns_accepted': ['ReturnsAccepted', 'ReturnsNotAccepted'],
                'refund_options': ['MoneyBack', 'MoneyBackOrExchange', 'MoneyBackOrReplacement'],
                'returns_within': ['Days_14', 'Days_30', 'Days_60'],
                'shipping_cost_paid_by': ['Buyer', 'Seller']
            }
        }

    def upload_photo(self, image_path: str) -> Optional[str]:
        """Upload a photo to eBay and return the URL"""
        xml_request = f"""<?xml version="1.0" encoding="utf-8"?>
    <UploadSiteHostedPicturesRequest xmlns="urn:ebay:apis:eBLBaseComponents">
        <RequesterCredentials>
            <eBayAuthToken>{self.auth_token}</eBayAuthToken>
        </RequesterCredentials>
        <PictureName>{os.path.basename(image_path)}</PictureName>
    </UploadSiteHostedPicturesRequest>"""
    
        try:
            with open(image_path, 'rb') as img_file:
                files = {
                    'XML Payload': ('', xml_request, 'text/xml'),
                    'image': (os.path.basename(image_path), img_file, 'application/octet-stream')
                }
                
                response = requests.post(
                    'https://api.ebay.com/ws/api.dll',
                    files=files,
                    headers={
                        'X-EBAY-API-SITEID': self.site_id,
                        'X-EBAY-API-COMPATIBILITY-LEVEL': self.compatibility_level,
                        'X-EBAY-API-IAF-TOKEN': self.auth_token,
                        'X-EBAY-API-CALL-NAME': 'UploadSiteHostedPictures'
                    }
                )
                
                response_dict = xmltodict.parse(response.content)
                return response_dict.get('UploadSiteHostedPicturesResponse', {}).get('SiteHostedPictureDetails', {}).get('FullURL')
        except Exception as e:
            print(f"Error uploading image: {str(e)}")
            return None


    def get_end_reasons(self) -> List[str]:
        """Get valid ending reason codes"""
        xml_request = """<?xml version="1.0" encoding="utf-8"?>
    <GetItemRequest xmlns="urn:ebay:apis:eBLBaseComponents">
        <RequesterCredentials>
            <eBayAuthToken>{}</eBayAuthToken>
        </RequesterCredentials>
        <DetailLevel>ReturnAll</DetailLevel>
        <OutputSelector>EndItemResponseDetails</OutputSelector>
    </GetItemRequest>""".format(self.auth_token)

        response = self._make_request('GetItem', xml_request)
        print("Raw Response:", response)  # Let's see what we get back

    # Let's also hard-code the known working values for now
    known_reasons = [
        {'code': 'NotAvailable', 'description': 'Item is no longer available'},
        {'code': 'LostOrBroken', 'description': 'Item is lost or broken'},
        {'code': 'Incorrect', 'description': 'Listing contains errors'},
        {'code': 'CustomCode', 'description': 'Custom reason'}
    ]

    print("\nKnown working end reasons:")
    for reason in known_reasons:
        print(f"Code: {reason['code']}, Description: {reason['description']}")


if __name__ == "__main__":
    # Initialize the trader
    trader = EbayTrader(auth_token=access_data['access_token'])

    # Get all active listings
    listings = trader.get_active_listings()

    # # Create a new listing
    # new_item = {
    #     'Title': 'Test Item',
    #     'Description': 'This is a test item',
    #     'CategoryID': '11450',  # You'll need the correct category ID
    #     'Price': '29.99',
    #     'Quantity': '1',
    #     'PayPalEmail': 'your.paypal@email.com',
    #     'Location': 'London',
    #     'ShippingCost': '3.99'
    # }
    # create_result = trader.create_listing(new_item)

    # # Update a listing
    # updates = {
    #     'Title': 'Updated Title',
    #     'Price': '39.99'
    # }
    # update_result = trader.update_listing('ITEM_ID', updates)

    # # End a listing
    # end_result = trader.end_listing('ITEM_ID')

    # # Mark as sold elsewhere
    # sold_result = trader.mark_as_sold('ITEM_ID')

ModuleNotFoundError: No module named 'xmltodict'

In [18]:
# Usage:
valid_reasons = trader.get_end_reasons()
print("Valid ending reasons:")
for reason in valid_reasons:
    print(f"Code: {reason['code']}, Description: {reason['description']}")

valid_reasons

NameError: name 'trader' is not defined

In [19]:
# requirements = trader.get_listing_requirements(category_id)
# print("\nMandatory fields:", requirements['mandatory_fields'])

# specifics = trader.get_category_specifics(category_id)
# print("\nRequired item specifics:", specifics)

# Example usage with photos
new_listing = {
    'Title': 'Apple iPhone 7 - 32GB - Graphite',
    'Description': 'Used iPhone 7 in good condition',
    'CategoryID': '9355',
    'Price': '329.99',
    'Quantity': '1',
    'ConditionID': '3000',  # Used
    'Location': 'London',
    'Currency': 'GBP',
    'ShippingCost': '2.99',
    'photos': ['https://imgs.search.brave.com/kjobBP0SIaMgF_mQ-FEpVrHtPaPYLdfjXr8K74Q1Q8k/rs:fit:860:0:0:0/g:ce/aHR0cHM6Ly90ZWNo/cHJvcy5jby51ay9j/ZG4vc2hvcC9maWxl/cy84YTg4OWRlZi1j/NDBlLTQ2ZjYtYmRk/OS0zMDA3YWZhMTZh/NTMtM18wZGUxYmIy/OS0yYWI3LTQ1ZmMt/YjhlOC1iNDBjMDJk/YTg3MTkuanBnP3Y9/MTcxNjM2MzI2MyZ3/aWR0aD0xMDI0', 
               'https://cdn.idealo.com/folder/Product/5113/0/5113043/s3_produktbild_gross/apple-iphone-7-128gb-rose-gold.jpg']  # Need to provide actual photo URLs
}


result = trader.create_listing(new_listing)
result

NameError: name 'trader' is not defined

In [217]:
# Get basic listing info only
basic_listings = trader.get_active_listings(include_details=False)

# Get full details including categories
detailed_listings = trader.get_active_listings(include_details=True)

basic_listings


Full Shipping Details from existing item:
{'ApplyShippingDiscount': 'false', 'CalculatedShippingRate': {'WeightMajor': {'@measurementSystem': 'Metric', '@unit': 'kg', '#text': '0'}, 'WeightMinor': {'@measurementSystem': 'Metric', '@unit': 'gm', '#text': '0'}}, 'SalesTax': {'SalesTaxPercent': '0.0', 'ShippingIncludedInTax': 'false'}, 'ShippingServiceOptions': [{'ShippingService': 'UK_myHermesDoorToDoorService', 'ShippingServiceCost': {'@currencyID': 'GBP', '#text': '2.99'}, 'ShippingServicePriority': '1', 'ExpeditedService': 'false', 'ShippingTimeMin': '2', 'ShippingTimeMax': '3'}, {'ShippingService': 'UK_CollectInPerson', 'ShippingServiceCost': {'@currencyID': 'GBP', '#text': '0.0'}, 'ShippingServicePriority': '2', 'ExpeditedService': 'false'}], 'ShippingType': 'Flat', 'ThirdPartyCheckout': 'false', 'ShippingDiscountProfileID': '0', 'InternationalShippingDiscountProfileID': '0', 'SellerExcludeShipToLocationsPreference': 'true'}

Full Shipping Details from existing item:
{'ApplyShippin

[{'ItemID': '167279800027',
  'Title': 'Apple iPhone 7 - 32GB - Rose Gold (BT Mobile) A1778 (GSM)',
  'Price': 'GBP',
  'Amount': '429.99',
  'Quantity': '1',
  'QuantityAvailable': '1',
  'ListingStatus': None,
  'ViewItemURL': 'https://www.ebay.co.uk/itm/Apple-iPhone-7-32GB-Rose-Gold-BT-Mobile-A1778-GSM--/167279800027',
  'GalleryURL': 'https://i.ebayimg.com/images/g/0NIAAOSwMW9nmKxM/s-l140.jpg',
  'StartTime': '2025-01-28T10:07:43.000Z',
  'TimeLeft': 'P29DT22H26M11S',
  'ShippingCost': '2.99'},
 {'ItemID': '167281756063',
  'Title': 'Apple iPhone 7 - 32GB - Graphite',
  'Price': 'GBP',
  'Amount': '329.99',
  'Quantity': '1',
  'QuantityAvailable': '1',
  'ListingStatus': None,
  'ViewItemURL': 'https://www.ebay.co.uk/itm/Apple-iPhone-7-32GB-Graphite-/167281756063',
  'GalleryURL': 'https://imgs.search.brave.com/kjobBP0SIaMgF_mQ-FEpVrHtPaPYLdfjXr8K74Q1Q8k/rs:fit:860:0:0:0/g:ce/aHR0cHM6Ly90ZWNo/cHJvcy5jby51ay9j/ZG4vc2hvcC9maWxl/cy84YTg4OWRlZi1j/NDBlLTQ2ZjYtYmRk/OS0zMDA3YWZhMTZh/NTMt

In [222]:
result = trader.end_listing(item_id="167281756063", reason="NotAvailable")
result

{'EndItemResponse': {'@xmlns': 'urn:ebay:apis:eBLBaseComponents',
  'Timestamp': '2025-01-29T11:50:11.008Z',
  'Ack': 'Success',
  'Version': 'E1397',
  'Build': 'E1397_UNI_API5_19110902_R1',
  'HardExpirationWarning': '2025-01-29 18:56:54',
  'EndTime': '2025-01-29T11:50:10.887Z'}}

In [170]:
# Let's say you want to list in the Mobile Phones category
category_id = '9355'  # Mobile Phones category

# Get all requirements and options
conditions = trader.get_condition_values(category_id)
print("Available conditions:")
for condition in conditions:
    print(f"ID: {condition['ID']} - {condition['DisplayName']}")

Available conditions:
ID: 1000 - New
ID: 1500 - Opened – never used
ID: 3000 - Used
ID: 7000 - For parts or not working


In [173]:
shipping_services = trader.get_shipping_services()
# print("\nAvailable shipping services:")
# for service in shipping_services:
#     print(f"ID: {service['ServiceID']} - {service['Description']}")

In [206]:
# Let's try it with your existing item ID from earlier
item_id = "167279800027"  # Your successful listing ID
details = trader.get_item_details(item_id)


Full Shipping Details from existing item:
{'ApplyShippingDiscount': 'false', 'CalculatedShippingRate': {'WeightMajor': {'@measurementSystem': 'Metric', '@unit': 'kg', '#text': '0'}, 'WeightMinor': {'@measurementSystem': 'Metric', '@unit': 'gm', '#text': '0'}}, 'SalesTax': {'SalesTaxPercent': '0.0', 'ShippingIncludedInTax': 'false'}, 'ShippingServiceOptions': [{'ShippingService': 'UK_myHermesDoorToDoorService', 'ShippingServiceCost': {'@currencyID': 'GBP', '#text': '2.99'}, 'ShippingServiceAdditionalCost': {'@currencyID': 'GBP', '#text': '0.0'}, 'ShippingServicePriority': '1', 'ExpeditedService': 'false', 'ShippingTimeMin': '2', 'ShippingTimeMax': '3'}, {'ShippingService': 'UK_CollectInPerson', 'ShippingServiceCost': {'@currencyID': 'GBP', '#text': '0.0'}, 'ShippingServiceAdditionalCost': {'@currencyID': 'GBP', '#text': '0.0'}, 'ShippingServicePriority': '2', 'ExpeditedService': 'false'}], 'ShippingType': 'Flat', 'ThirdPartyCheckout': 'false', 'ShippingDiscountProfileID': '0', 'Interna